In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [2]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

That's great! Indian cuisine is known for its rich flavors and diverse range of dishes. Here are a few popular Indian recipes you can try:

1. Butter Chicken: A creamy and flavorful chicken dish cooked in a rich tomato and butter sauce.

2. Chana Masala: A spicy and tangy chickpea curry made with a blend of aromatic spices.

3. Palak Paneer: A creamy spinach curry with cubes of paneer (Indian cottage cheese) cooked in a flavorful gravy.

4. Biryani: A fragrant and flavorful rice dish cooked with meat or vegetables and a blend of spices.

5. Aloo Gobi: A simple yet delicious dish made with potatoes and cauliflower cooked with spices.

6. Dal Tadka: A comforting lentil curry tempered with aromatic spices.

These are just a few examples, but there are countless other Indian dishes you can explore. Enjoy your cooking!As a vegetarian chef specialized in making traditional recipes vegetarian, I can suggest some alternatives for the non-vegetarian ingredients in the Indian recipes you mention

AIMessageChunk(content='As a vegetarian chef specialized in making traditional recipes vegetarian, I can suggest some alternatives for the non-vegetarian ingredients in the Indian recipes you mentioned:\n\n1. Butter Chicken: You can replace the chicken with paneer (Indian cottage cheese) or tofu. Marinate the paneer or tofu in a mixture of yogurt and spices before cooking it in the buttery tomato sauce.\n\n2. Chana Masala: This dish is already vegetarian! Just make sure to use vegetable oil instead of ghee for a fully vegetarian version.\n\n3. Palak Paneer: Instead of paneer, you can use tofu or boiled and cubed potatoes. Both options will absorb the flavors of the creamy spinach gravy beautifully.\n\n4. Biryani: For a vegetarian biryani, you can substitute the meat with mixed vegetables like carrots, peas, potatoes, and cauliflower. Layer the spiced vegetables with fragrant basmati rice and cook them together for a delicious vegetarian version.\n\n5. Aloo Gobi: This dish is already ve